## R: Differential expression analysis, Monocle/trajectory analysis, cluster 6 analysis

In [ ]:
library(dplyr)
library(Seurat)
library(patchwork)
library(tidyverse)
library(SeuratWrappers)
library(monocle3)
library(biomaRt)
library(ggrepel)

In [ ]:
data = readRDS("seurat_final.rds")
data

In [ ]:
DimPlot(data, reduction = "umap", group.by ="seurat_clusters", pt.size = 0.1, cols =
          c('0' = '#0078B9', '1' = '#FF7C00', '2'='#00A163', '3'='#E7001A', '4'='#B62BFE', '5'='#92563A', 
            '6'='#F56DC2','7'='#ADC152', '8'='#00C1D3', '9'='#A6C9ED', '10'='#FFB86B', '11'='#81E580', 
            '12'='#FF9191')) + 
  ggtitle("") + xlab("UMAP 1") + ylab("UMAP 2")
table(data@meta.data$patient)

In [ ]:
DimPlot(data, reduction = "umap", group.by ="type", pt.size = 0.1, 
        cols = c('Neonate'='#FF7300','Child'='#006EB6')) + ggtitle("") + 
  xlab("UMAP 1") + ylab("UMAP 2")

In [ ]:
counts <- group_by(data@meta.data, type, seurat_clusters) %>% summarise(count = n())
counts$count_norm <- ave(counts$count, counts$type, FUN=function(x) x/sum(x))
ggplot(counts, aes(type, count_norm, fill = seurat_clusters)) + 
                         geom_bar(stat = 'identity', colour="black", size = 0.1) + 
                         theme_classic() + theme(text = element_text(size = 16),legend.position = "none") + 
                         xlab("") + ylab("Fraction of Cells") +
                         scale_fill_manual("legend" ,values = c('0' = '#0078B9', '1' = '#FF7C00', '2'='#00A163', 
                                                                '3'='#E7001A', '4'='#B62BFE', '5'='#92563A', 
                                                                '6'='#F56DC2', '7'='#ADC152', '8'='#00C1D3', 
                                                                '9'='#A6C9ED', '10'='#FFB86B', '11'='#81E580', 
                                                                '12'='#FF9191'))

In [ ]:
DotPlot(data, features = c("PDGFRA","DDR2","COL1A1","POSTN",
                            "ACKR3","THY1","AXL","CD34",
                            "APOE","BMP4","ADM",
                            "WIF1","WNT5A","WNT16","DKK3","SFRP2","FRZB","AXIN2","PTN","TGFB3","CCL19","CXCL12",
                            "ELN","MFAP4","COL3A1","COL14A1","FBLN1","NPPC","CTGF","NOX4","ACTA2","PLOD2"
                         )) + 
  theme(axis.text.x = element_text(angle = 90,hjust=1)) + coord_flip()

In [ ]:
counts

### Get cluster markers

In [ ]:
markers = FindAllMarkers(data, logfc.threshold = 0.25, 
                        test.use = "wilcox", 
                        min.pct = 0.1, min.diff.pct = 0.1,
                        only.pos = TRUE)

In [ ]:
DotPlot(obj, features = c("ACTA2","IGFBP3","IGFBP5",
                         "SOX17","TCF4","SOX4",
                         "")) + 
  theme(axis.text.x = element_text(angle = 90,hjust=1)) + coord_flip()

# Bulk

In [ ]:
data@meta.data %>% head()

### PCA neonate vs. child

In [ ]:
sub_agg = AggregateExpression(data, slot = "scale.data", group.by = "patient", verbose = TRUE)
# png("PCApseudobulk.png", width = 2200, height = 2000, res = 300)
a = prcomp(sub_agg$SCT)$rotation %>% as.data.frame() %>% rownames_to_column("patient") 
right_join(a, data@meta.data[,c(3,4)] %>% group_by(patient,type) %>% summarise(count = n())) %>% 
    ggplot(aes(x = PC1, y = PC2, label = patient)) + 
        geom_point(aes(color = type), size = 5) + theme_classic() +
        scale_color_manual(values = c('Neonate' = '#F47C20', 'Child' = '#2E7BBD')) + 
        theme(text = element_text(size = 20))       
# dev.off()

## nCPC vs cCPC each cluster

In [ ]:
rna2 = data
rna2$edgeR = paste0(rna2@meta.data$patient, "_", rna2@meta.data$seurat_clusters)
columns = as.character(levels(factor(rna2@meta.data$edgeR)))
columns[1:5]
length(columns)

var = rna2
Idents(var) = "edgeR"
countmatrix = data.frame(rep(0,36601))
for (i in 1:length(columns)){
    print(columns[i])
    var_subset = subset(var, idents = columns[i])
    print(dim(var_subset))
    var_umi = GetAssayData(var_subset, slot = "counts")
    var_umi_mat = as.matrix(var_umi)
    var_sums = rowSums(var_umi_mat)
    countmatrix = cbind(countmatrix, var_sums)}
countmtx = countmatrix[,2:ncol(countmatrix)] 
colnames(countmtx) = columns
countmtx%>% tail()

In [ ]:
meta = rna2@meta.data[,3:6] %>% group_by_all() %>% tally()
meta

In [ ]:
p = str_extract(colnames(countmtx), "[^_]+")

In [ ]:
patient = str_extract(colnames(countmtx), "[^_]+")
passage = c(); for(j in 1:length(patient)){passage = c(passage,unique(meta[meta$passage == patient[j],3])[[1]])}
patient

In [ ]:
left_join(as.data.frame(patient),meta) %>% head()

In [ ]:
library(edgeR)
for(i in 0:12){
    cluster = i 
    pseudo = countmtx[,word(colnames(countmtx), 2, sep = "_") == cluster]
    patient = str_extract(colnames(pseudo), "[^_]+")
    passage = left_join(as.data.frame(patient),meta)$passage
    group = left_join(as.data.frame(patient),meta)$type
    #disease = left_join(as.data.frame(patient),meta)$disease
    y = DGEList(counts = pseudo, group = group)
    design = model.matrix(~passage+group)
    rownames(design) = colnames(y)
    keep = filterByExpr(y)
    DGEy = y[keep, , keep.lib.sizes=FALSE]
    table(keep)
    y = calcNormFactors(y)
    y = estimateDisp(y, design)
    fitlr = glmFit(y, design) #Fit negative binomial generalized log-linear model to read counts for each gene 
    lr =  glmLRT(fitlr) #
    fdr = p.adjust(lr$table$PValue,method="BH")
    tab = topTags(lr,n = nrow(lr))
    write.csv(tab, paste0("cluster",i,"_DEGs.csv"))
    
    tab$table = tab$table %>% rownames_to_column("hgnc_symbol")
    tab$table$p = as.factor(ifelse(tab$table$FDR < 0.05, "p", "no"))
    tab$table$dir = as.factor(ifelse((tab$table$FDR < 0.05 & abs(tab$table$logFC)>1), 
                                       if_else(tab$table$logFC > 1, "Neonate","Child"), "NS"))
    print(table(tab$table$dir))
    tab$table$RNA = ""
    #get protein_coding only to label
    mart = useMart("ENSEMBL_MART_ENSEMBL", host = "useast.ensembl.org")
    mart = useDataset("hsapiens_gene_ensembl", mart)
    pc = left_join(tab$table, getBM(mart = mart,attributes = c("hgnc_symbol","gene_biotype"),
                                            filter = "hgnc_symbol",
                                            values = tab$table$hgnc_symbol,uniqueRows=TRUE)) %>% 
                                                filter(gene_biotype == "protein_coding")
    toptags = pc %>% group_by(dir) %>% filter(dir!="NS") %>% 
                slice_max(n = 10, order_by = -log10(FDR))
    #plot
    tab$table$RNA[which(tab$table$hgnc_symbol %in% toptags$hgnc_symbol)] = tab$table$hgnc_symbol[tab$table$hgnc_symbol %in% toptags$hgnc_symbol]
    
    assign(paste0("cluster",i,"p"),tab$table)
    }

In [ ]:
head(cluster0p)

In [ ]:
png("cluster7.png",res = 200, height = 1500, width = 1500)
cluster7p %>% ggplot(aes(x = logFC, y = -log10(FDR), label = RNA)) + 
        geom_point(aes(color = dir)) +
        theme_classic() + theme(text = element_text(size = 14),legend.title=element_blank()) +
        geom_hline(yintercept = -log10(0.05), color = "grey") + 
        geom_vline(xintercept = c(-1,1), color = "grey") +
        scale_color_manual(values = c("Neonate" = "#F47C20", 'Child' = '#2E7BBD', "NS" = "black"))+
        geom_text_repel()+ggtitle("Cluster 7")+labs(caption = "Model: ~ passage + age group")
dev.off()

In [ ]:
c6 = subset(data, cells = colnames(data)[Idents(data)==6])

In [ ]:
dim(c6)

In [ ]:
#c6 = RunPCA(c6, verbose = FALSE)
#c6 = RunUMAP(c6, dims = 1:30, verbose = FALSE)
#c6 = FindNeighbors(c6, dims = 1:30, verbose = FALSE)
c6 = FindClusters(c6, resolution = 0.05, verbose = FALSE)
DimPlot(c6, reduction = "umap", label = TRUE)


In [ ]:
#png(paste0(maindir,"plots/Dotplot_Clusters_3.png"), width = 1800, height = 2200, res = 300)
DoHeatmap(c6 %>% FindVariableFeatures(nfeatures = 50), label = FALSE)

In [ ]:
DotPlot(c6,features = c("COL1A1","PDGFRA","THY1",
             "POSTN","PLOD2","ACTA2",
            "WIF1","WNT5A","DKK3","CXCL12",# Wntx
            "COL3A1","COL14A1","FBLN1"       # F-trans
            
                         )) + 
  theme(axis.text.x = element_text(angle = 90,hjust=1)) + coord_flip()

# Trajectory Analysis

In [ ]:
#trace("calculateLW", edit = T, where = asNamespace("monocle3"))
cds <- as.cell_data_set(data)
cds <- estimate_size_factors(cds)
rowData(cds)$gene_name <- rownames(cds)
rowData(cds)$gene_short_name <- rowData(cds)$gene_name
cds <- cluster_cells(cds = cds, reduction_method = "UMAP")
cds <- learn_graph(cds, learn_graph_control=list(ncenter=500))

In [ ]:
cds

# Coexpressed genes along trajectories

In [ ]:
trace("calculateLW", edit = T, where = asNamespace("monocle3"))

In [ ]:
de_res <- graph_test(cds, neighbor_graph = "principal_graph", cores = 48)
#saveRDS(de_res, "de_res_09082022_revisions.rds")
head(de_res)

In [ ]:
library(data.table)
de_res_filt <- subset(de_res, q_value < 0.05)
de_res_dt <- data.table(de_res_filt)
plot_cells(cds, genes=as.matrix(de_res_dt[order(-morans_I),][1:20,"gene_name"])[,1],
           show_trajectory_graph=FALSE,
           label_cell_groups=FALSE,
           label_leaves=FALSE)

# Use Monocle's built-in batch correction before finding gene modules

In [ ]:
cds <- preprocess_cds(cds)
cds <- align_cds(cds, alignment_group = "batch")
gene_module_df <- find_gene_modules(cds[rownames(de_res_filt),], resolution = c(10^seq(-6,-1)), cores = 48)b

In [ ]:
library(RColorBrewer)
cell_group_df <- tibble::tibble(cell=row.names(colData(cds)), 
                                cell_group=colData(cds)$seurat_clusters)
agg_mat <- aggregate_gene_expression(cds, gene_module_df, cell_group_df)
row.names(agg_mat) <- stringr::str_c("Module ", row.names(agg_mat))
pheatmap::pheatmap(agg_mat,
                   scale="column", clustering_method="ward.D2")b

In [ ]:
library(ComplexHeatmap)
clust_mod <- agg_mat
cell_group_df <- tibble::tibble(cell=row.names(colData(cds)), 
                                cell_group=colData(cds)$type)
age_mod <- aggregate_gene_expression(cds, gene_module_df, cell_group_df)
row.names(age_mod) <- stringr::str_c("Module ", row.names(age_mod))

scaled_mat = scale(as.matrix(clust_mod))

Child <- circlize::colorRamp2(seq(min(age_mod[,1]), max(age_mod[,1]), length = 7), brewer.pal(n = 7, name = "Blues"))
Neonate <- circlize::colorRamp2(seq(min(age_mod[,2]), max(age_mod[,2]), length = 7), brewer.pal(n = 7, name = "Oranges"))

colnames(scaled_mat) <- stringr::str_c("Cluster ", colnames(scaled_mat))

In [ ]:
color = list(Child = Child,
            Neonate = Neonate)
row_ha = rowAnnotation(Child = age_mod[,1], Neonate = age_mod[,2], col=color)
col_ha = HeatmapAnnotation(Child = anno_barplot(counts[1:13,4], ylim = c(0,0.35)), Neonate = anno_barplot(counts[14:26,4], ylim = c(0,0.35)))
Heatmap(scaled_mat, 
        right_annotation = row_ha, 
        top_annotation = col_ha, 
        col=colorRampPalette(rev(brewer.pal(n = 7, name = "RdYlBu")))(100),
        name="Module\nExpression")